## Libraries

In [1]:
import chatbot
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
from dotenv import load_dotenv
from openai import OpenAI
from collections import Counter

load_dotenv()
client = OpenAI()

index, metadata = chatbot.load_faiss_index(chatbot.VECTOR_DIR)
print("Index and metadata loaded.")

Index and metadata loaded.


In [2]:
with open("evaluation_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [3]:
def json_to_documents(data: dict):
    documents = []

    for source, content in data.items():
        qa_list = content.get("perguntas_respostas", [])

        for item in qa_list:
            documents.append({
                "question": item.get("pergunta", ""),
                "answer": item.get("resposta", ""),
                "source": source
            })

    return documents


In [4]:
documents = json_to_documents(data)
documents

[{'question': 'O que é o Alentejo 2030?',
  'answer': 'O Alentejo 2030 é um programa que transforma vidas impulsionando a inovação, a sustentabilidade e o crescimento da região.',
  'source': 'alentejo2030'},
 {'question': 'Quais são alguns dos eventos programados para o Alentejo 2030?',
  'answer': 'Alguns eventos programados incluem um workshop sobre Prospetivas e Políticas Públicas em 15 de dezembro de 2025 e uma conferência sobre Inteligência Artificial em 27 de novembro de 2025.',
  'source': 'alentejo2030'},
 {'question': 'Quando foi aprovada a reprogramação do Alentejo 2030 pela Comissão Europeia?',
  'answer': 'A reprogramação do Alentejo 2030 foi aprovada pela Comissão Europeia no dia 15 de dezembro de 2025.',
  'source': 'alentejo2030'},
 {'question': 'Quais são algumas das novas prioridades específicas integradas no Alentejo 2030?',
  'answer': 'As novas prioridades incluem a Defesa, Água, Habitação a preços acessíveis e sustentável, e Competências para a descarbonização.',


# Evaluation Dataset

In [9]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.evaluation import DatasetGenerator
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.openai import OpenAI

import random

In [10]:
# 1. Load documents
documents = SimpleDirectoryReader("data/04_metadata").load_data()

# 2. Define LLM
llm = OpenAI(model="gpt-4o-mini", temperature=0.2)

In [11]:
node_parser = SimpleNodeParser.from_defaults()
nodes = node_parser.get_nodes_from_documents(documents)

In [12]:
prompt = """
    Escreve sempre em Português de Portugal
    """

In [13]:
dataset_generator = DatasetGenerator(
    nodes=nodes,
    llm=llm,
    num_questions_per_chunk=1,
    question_gen_query=prompt,
    show_progress = True
)

/var/folders/mm/fxsq_1490x9dd2w76tqvt3kr0000gn/T/ipykernel_25824/1493583760.py:1: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  dataset_generator = DatasetGenerator(


In [14]:
eval_dataset = dataset_generator.generate_dataset_from_nodes()

  0%|                                                  | 0/3936 [00:00<?, ?it/s]
KeyboardInterrupt



In [57]:
questions = rag_dataset.questions
questions

['Quais são os principais desafios que o envelhecimento demográfico representa para a medicina, especialmente na área da ortopedia?',
 'Como as limitações dos implantes ósseos convencionais afetam as intervenções cirúrgicas em indivíduos idosos?',
 'O que é o projeto RESET_Bone_Ageing_2 e qual é o seu objetivo na área dos dispositivos médicos?',
 'Quem são os principais participantes do projeto RESET_Bone_Ageing_2?',
 'De que forma a utilização de dispositivos médicos em ortopedia tem evoluído nas últimas décadas?',
 'Quais são as implicações do aumento da utilização de dispositivos médicos para o tratamento de pacientes idosos?',
 'Como a integração de implantes ósseos em tecido ósseo envelhecido pode ser melhorada?',
 'Quais são os tópicos abordados nas notícias relacionadas com o envelhecimento e dispositivos médicos?',
 'Que papel desempenham as universidades na pesquisa e desenvolvimento de soluções para os desafios do envelhecimento na ortopedia?',
 'Como a prática médica tem mud

In [58]:
len(questions)

995

## RAGAS

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import time

# ---------------- Embedding Helper ----------------
def get_embedding(text: str):
    """
    Returns embedding using your OpenAI client
    """
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return np.array(response.data[0].embedding, dtype=np.float32).reshape(1, -1)

# ---------------- Core Metrics ----------------
def f1_score(pred, ref):
    if not pred: return 0.0
    pred_tokens = pred.split()
    ref_tokens = ref.split()
    common = Counter(pred_tokens) & Counter(ref_tokens)
    num_same = sum(common.values())
    if num_same == 0: return 0.0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(ref_tokens)
    return 2 * precision * recall / (precision + recall)

def exact_match(pred, ref):
    return int(pred.strip() == ref.strip())

def conciseness_score(generated, reference):
    return len(reference.split()) / max(1, len(generated.split()))

def semantic_similarity_score(generated, references):
    """
    Computes max semantic similarity between generated answer and reference answers
    """
    gen_emb = get_embedding(generated)
    ref_embs = np.vstack([get_embedding(ref) for ref in references])
    sims = cosine_similarity(gen_emb, ref_embs)[0]
    return float(np.max(sims))


# ---------------- Embedding-based Retrieval Success ----------------
def retrieval_success_semantic(retrieved_docs, reference):
    """
    Returns the maximum semantic similarity between the reference and retrieved docs.
    If no docs are retrieved, returns 0.0.
    """
    if not retrieved_docs:
        return 0.0

    ref_emb = get_embedding(reference)
    max_sim = 0.0

    for doc in retrieved_docs:
        doc_text = doc.get("answer", "").strip()
        if not doc_text:
            continue
        doc_emb = get_embedding(doc_text)
        sim = cosine_similarity(ref_emb, doc_emb)[0][0]
        if sim > max_sim:
            max_sim = sim

    return float(max_sim)


# ---------------- Embedding-based Grounding ----------------
def grounding_score_semantic(generated, retrieved_docs):
    """
    Fraction of generated answer semantically supported by retrieved docs
    """
    if not retrieved_docs:
        return 0.0

    gen_emb = get_embedding(generated)
    doc_embs = np.vstack([get_embedding(doc.get("answer","")) for doc in retrieved_docs])
    sims = cosine_similarity(gen_emb, doc_embs)[0]
    return float(np.max(sims)) if len(sims) > 0 else 0.0



**Pipeline**

In [42]:
def evaluate_rag_semantic(dataset, index, metadata, k=5, model="gpt-4o-mini"):
    results = []

    for item in dataset:
        question = item["question"]
        reference = item["answer"]
        source = item.get("source", "unknown")

        # Generate answer
        start_time = time.time()
        generated_answer, retrieved_docs = chatbot.answer(question, index, metadata, k=k, model=model)
        latency = time.time() - start_time

        if isinstance(generated_answer, tuple):
            generated_answer = generated_answer[0]

        # Metrics
        retrieval_success = retrieval_success_semantic(retrieved_docs, reference)
        grounding = grounding_score_semantic(generated_answer, retrieved_docs)
        f1 = f1_score(generated_answer, reference)
        em = exact_match(generated_answer, reference)
        conciseness = conciseness_score(generated_answer, reference)
        semantic_sim = semantic_similarity_score(generated_answer, [reference])

        results.append({
            "Question": question,
            "Reference Answer": reference,
            "Source": source,
            "Generated Answer": generated_answer,
            "Retrieval Success (semantic)": retrieval_success,
            "Grounding (semantic)": grounding,
            "F1": f1,
            "Exact Match": em,
            "Conciseness": conciseness,
            "Semantic Similarity": semantic_sim,
            "Latency (s)": latency
        })

    df = pd.DataFrame(results)
    return df


In [43]:
df_results = evaluate_rag_semantic(documents, index, metadata, k=5)


Assistente: O Alentejo 2030 é um programa regional aprovado pela Comissão Europeia para o período de 2021-2027, que tem como objetivo impulsionar a inovação, a sustentabilidade e o crescimento da região do Alentejo. Este programa disponibiliza fundos europeus para apoiar diversas iniciativas e projetos que visam o desenvolvimento económico e social da região, contribuindo para a melhoria da qualidade de vida dos seus habitantes. (Fontes: alentejo2030, portugal2030)

Assistente: O documento não especifica eventos concretos programados para o Alentejo 2030. No entanto, menciona que haverá um evento que celebra projetos transformadores em Portugal, incluindo iniciativas do Alentejo 2030, com debates, apresentações e a cerimónia de entrega dos Prémios dos Fundos Europeus. Para informações mais detalhadas sobre eventos específicos, recomenda-se consultar fontes oficiais ou o site do Alentejo 2030. (Fontes: alentejo2030, portugal2030)

Assistente: A reprogramação do Alentejo 2030 foi aprova

In [45]:
df_results.describe()

,Retrieval Success (semantic),Grounding (semantic),F1,Exact Match,Conciseness,Semantic Similarity,Latency (s)
count,30.0,30.000000,30.000000,30.0,30.000000,30.000000,30.000000
mean,0.0,0.124298,0.410946,0.0,0.498041,0.792829,3.565881
std,0.0,0.033876,0.222132,0.0,0.242503,0.133917,0.827905
min,0.0,0.000000,0.098039,0.0,0.217822,0.452787,2.644735
25%,0.0,0.114374,0.239701,0.0,0.289680,0.709155,2.965225
50%,0.0,0.131351,0.346646,0.0,0.475250,0.839655,3.356700
75%,0.0,0.143351,0.585294,0.0,0.620423,0.877036,4.034765
max,0.0,0.200045,0.918919,0.0,1.277778,0.971786,6.035443


In [51]:
df_results[['Question', "Reference Answer", "Generated Answer", "F1"]]

,Question,Reference Answer,Generated Answer,F1
0,O que é o Alentejo 2030?,"O Alentejo 2030 é um programa que transforma vidas impulsionando a inovação, a sustentabilidade e o crescimento da região.","O Alentejo 2030 é um programa regional aprovado pela Comissão Europeia para o período de 2021-2027, que tem como objetivo impulsionar a inovação, a sustentabilidade e o crescimento da região do Alentejo. Este programa disponibiliza fundos europeus para apoiar diversas iniciativas e projetos que visam o desenvolvimento económico e social da região, contribuindo para a melhoria da qualidade de vida dos seus habitantes. (Fontes: alentejo2030, portugal2030)",0.352941
1,Quais são alguns dos eventos programados para o Alentejo 2030?,Alguns eventos programados incluem um workshop sobre Prospetivas e Políticas Públicas em 15 de dezembro de 2025 e uma conferência sobre Inteligência Artificial em 27 de novembro de 2025.,"O documento não especifica eventos concretos programados para o Alentejo 2030. No entanto, menciona que haverá um evento que celebra projetos transformadores em Portugal, incluindo iniciativas do Alentejo 2030, com debates, apresentações e a cerimónia de entrega dos Prémios dos Fundos Europeus. Para informações mais detalhadas sobre eventos específicos, recomenda-se consultar fontes oficiais ou o site do Alentejo 2030. (Fontes: alentejo2030, portugal2030)",0.153846
2,Quando foi aprovada a reprogramação do Alentejo 2030 pela Comissão Europeia?,A reprogramação do Alentejo 2030 foi aprovada pela Comissão Europeia no dia 15 de dezembro de 2025.,"A reprogramação do Alentejo 2030 foi aprovada pela Comissão Europeia no dia 15 de dezembro de 2025. (Fontes: alentejo2030, portugal2030)",0.918919
3,Quais são algumas das novas prioridades específicas integradas no Alentejo 2030?,"As novas prioridades incluem a Defesa, Água, Habitação a preços acessíveis e sustentável, e Competências para a descarbonização.","Algumas das novas prioridades específicas integradas no Alentejo 2030 incluem:\n\n1. **Habitação acessível** - Promover a fixação populacional e a justiça social.\n2. **Gestão e eficiência da água** - Reforçar a sustentabilidade e resiliência do território.\n\nEstas prioridades visam fortalecer a resposta da região a desafios económicos, sociais, ambientais e territoriais, contribuindo para um desenvolvimento mais sustentável e inclusivo. (Fontes: alentejo2030)",0.202532
4,Qual é o objetivo da reprogramação do Alentejo 2030?,"O objetivo da reprogramação é alinhar os fundos europeus com novas prioridades estratégicas e desafios emergentes da região, contribuindo para um desenvolvimento mais sustentável, inclusivo e resiliente do Alentejo.","O objetivo da reprogramação do Alentejo 2030 é fortalecer a resposta da região a desafios económicos, sociais, ambientais e territoriais atuais e futuros, promovendo um desenvolvimento mais sustentável, inclusivo e resiliente. A reprogramação também visa alinhar os fundos europeus com novas prioridades estratégicas e desafios emergentes da região. (Fontes: alentejo2030, portugal2030)",0.600000
5,Qual é o objetivo principal do Programa Algarve 2030?,"O Programa Algarve 2030 destina-se a promover a competitividade e diversificação da economia e a sustentabilidade regional, valorizando o território e as pessoas.","O objetivo principal do Programa Algarve 2030 é promover a competitividade, a diversificação da economia e a sustentabilidade da região do Algarve. O programa visa também valorizar o território e as pessoas, com foco na melhoria da qualidade de vida e na inclusão social. (Fontes: algarve2030, portugal2030)",0.514286
6,Quantas candidaturas foram recebidas até 28 de novembro de 2025 para o ciclo urbano da água?,Foram recebidas 25 candidaturas de Municípios e entidades gestoras do ciclo urbano da água.,"Até 28 de novembro de 2025, foram recebidas 25 candidaturas para o ciclo urbano da água, totalizando um investimento de mais de 100 milhões de euros. (Fontes: portugal2030, algarve2030)",0.325

In [49]:
pd.set_option('display.max_colwidth', None)

## Bot-Based

In [54]:
def eval_chat(question, expected_answer, actual_tuple):
    """
    Returns a similarity score 0-10
    """
    actual_answer, retrieved_docs = actual_tuple

    # Embedding-based similarity with the reference
    semantic_sim = semantic_similarity_score(actual_answer, [expected_answer])

    # Scale from 0-1 → 0-10
    score = int(round(semantic_sim * 10))
    return score


In [71]:
from tqdm import tqdm
import time
import pandas as pd

def bot_evaluation(dataset, index, metadata, k=5, model="gpt-4o-mini"):
    results = []

    for idx, item in enumerate(tqdm(dataset, desc="Evaluating RAG")):
        question = item["question"]
        expected_answer = item["answer"]
        source = item.get("source", "unknown")

        # --- Generate answer ---
        start_time = time.time()
        generated_answer, retrieved_docs = chatbot.answer(
            question,
            index,
            metadata,
            k=k,
            model=model
        )
        latency = time.time() - start_time

        if isinstance(generated_answer, tuple):
            generated_answer = generated_answer[0]

        # --- Compute BPT score (0-10) ---
        # You can include grounding if you want
        semantic_sim = semantic_similarity_score(generated_answer, [expected_answer])
        grounding = grounding_score_semantic(generated_answer, retrieved_docs)
        # Weighted combination (optional)
        bpt_score = int(round(0.7 * semantic_sim * 10 + 0.3 * grounding * 10))

        results.append({
            "Question_ID": idx,
            "Source": source,
            "Question": question,
            "Expected Answer": expected_answer,
            "Generated Answer": generated_answer,
            "BPT Score": bpt_score,
            "Response Time (s)": latency
        })

    return pd.DataFrame(results)


In [72]:
results = bot_evaluation(documents, index, metadata)

Evaluating RAG:   0%|                                    | 0/30 [00:00<?, ?it/s]


Assistente: O Alentejo 2030 é um programa regional aprovado pela Comissão Europeia para o período de 2021-2027, que visa promover a inovação, a sustentabilidade e o crescimento da região do Alentejo. Através deste programa, a região dispõe de uma dotação global significativa de fundos europeus, destinados a apoiar diversas áreas e iniciativas que contribuam para o desenvolvimento regional. (Fontes: alentejo2030, portugal2030)


Evaluating RAG:   3%|▉                           | 1/30 [00:09<04:33,  9.44s/it]


Assistente: O documento não especifica eventos concretos programados para o Alentejo 2030. No entanto, menciona que haverá um evento que celebra projetos transformadores em Portugal, incluindo iniciativas do Alentejo 2030, com debates, apresentações e a cerimónia de entrega dos Prémios dos Fundos Europeus. Para informações mais detalhadas sobre eventos específicos, recomendo consultar o site oficial do Alentejo 2030. (Fontes: alentejo2030, portugal2030)


Evaluating RAG:   7%|█▊                          | 2/30 [00:15<03:33,  7.62s/it]

...

Evaluating RAG:  93%|█████████████████████████▏ | 28/30 [03:29<00:12,  6.44s/it]


Assistente: O objetivo do projeto CustomNiTi é desenvolver placas e distratores ósseos personalizados que melhoram a eficiência e o conforto no tratamento de fraturas ósseas. Este projeto, cofinanciado pelo Programa COMPETE 2030, visa inovar o setor de dispositivos médicos, proporcionando soluções clínicas eficazes e económicas. (Fontes: portugal2030)


Evaluating RAG:  97%|██████████████████████████ | 29/30 [03:35<00:06,  6.30s/it]


Assistente: Você pode entrar em contato com a Linha dos Fundos para esclarecer dúvidas sobre o Portugal 2030 através dos seguintes meios:

- **Telefone**: (+351) 289 895 232/37 (custo gratuito para chamadas de Portugal)
- **E-mail**: linhadosfundos@linhadosfundos.pt

A Linha dos Fundos está disponível todos os dias úteis, entre as 9h e as 18h. (Fontes: portugal2030)


Evaluating RAG: 100%|███████████████████████████| 30/30 [03:41<00:00,  7.37s/it]


In [73]:
results.describe()

,Question_ID,BPT Score,Response Time (s)
count,30.000000,30.000000,30.000000
mean,14.500000,5.866667,4.013028
std,8.803408,0.899553,1.232095
min,0.000000,4.000000,2.097220
25%,7.250000,5.250000,3.123637
50%,14.500000,6.000000,3.726955
75%,21.750000,6.000000,4.770928
max,29.000000,7.000000,6.723830


In [75]:
results[results['BPT Score'] < 6]

,Question_ID,Source,Question,Expected Answer,Generated Answer,BPT Score,Response Time (s)
1,1,alentejo2030,Quais são alguns dos eventos programados para o Alentejo 2030?,Alguns eventos programados incluem um workshop sobre Prospetivas e Políticas Públicas em 15 de dezembro de 2025 e uma conferência sobre Inteligência Artificial em 27 de novembro de 2025.,"O documento não especifica eventos concretos programados para o Alentejo 2030. No entanto, menciona que haverá um evento que celebra projetos transformadores em Portugal, incluindo iniciativas do Alentejo 2030, com debates, apresentações e a cerimónia de entrega dos Prémios dos Fundos Europeus. Para informações mais detalhadas sobre eventos específicos, recomendo consultar o site oficial do Alentejo 2030. (Fontes: alentejo2030, portugal2030)",4,3.890048
3,3,alentejo2030,Quais são algumas das novas prioridades específicas integradas no Alentejo 2030?,"As novas prioridades incluem a Defesa, Água, Habitação a preços acessíveis e sustentável, e Competências para a descarbonização.","Algumas das novas prioridades específicas integradas no Alentejo 2030 incluem:\n\n1. **Habitação acessível** - Promovendo a fixação populacional e a justiça social.\n2. **Gestão e eficiência da água** - Reforçando a sustentabilidade e resiliência do território.\n\nEstas prioridades visam fortalecer a resposta da região a desafios económicos, sociais, ambientais e territoriais, contribuindo para um desenvolvimento mais sustentável e inclusivo. (Fontes: alentejo2030)",5,4.053267
6,6,algarve2030,Quantas candidaturas foram recebidas até 28 de novembro de 2025 para o ciclo urbano da água?,Foram recebidas 25 candidaturas de Municípios e entidades gestoras do ciclo urbano da água.,"Até 28 de novembro de 2025, foram recebidas 25 candidaturas para o ciclo urbano da água, totalizando um investimento superior a 100 milhões de euros. (Fontes: portugal2030, algarve2030)",5,3.032607
7,7,algarve2030,Qual é o valor total do investimento das candidaturas relacionadas ao ciclo urbano da água?,O investimento total das candidaturas é de mais de 100 milhões de euros.,"O valor total do investimento das candidaturas relacionadas ao ciclo urbano da água é superior a 100 milhões de euros. (Fontes: portugal2030, algarve2030)",5,2.351102
8,8,algarve2030,Quais são algumas das intervenções estratégicas previstas no Programa Algarve 2030?,"As intervenções incluem a redução de perdas nas redes urbanas de água, reabilitação de infraestruturas obsoletas, prevenção e mitigação da intrusão salina, e reutilização de águas residuais tratadas.","Algumas das intervenções estratégicas previstas no Programa Algarve 2030 incluem:\n\n1. **Promoção da competitividade das empresas** - Através de apoio à inovação e digitalização.\n2. **Sustentabilidade ambiental** - Focando na gestão eficiente dos recursos naturais.\n3. **Valorização do território** - Incentivando o desenvolvimento urbano sustentável e a reabilitação de áreas urbanas.\n4. **Qualidade de vida** - Melhorando os serviços públicos e a acessibilidade.\n\nEstas intervenções visam contribuir para um desenvolvimento equilibrado e sustentável da região do Algarve. (Fontes: algarve2030, portugal2030)",4,5.081694
12,12,centro2030,Qual é o valor do financiamento destinado à habitação no âmbito da reprogramação do CENTRO 2030?,O financiamento destinado à habitação (social e a preços acessíveis) é de quase 200 milhões de euros.,"O valor do financiamento destinado à habitação no âmbito da reprogramação do CENTRO 2030 é de quase 200 milhões de euros. Este investimento visa ampliar o parque público de habitação, promovendo a renovação de edifícios devolutos ou vagos. (Fontes: centro2030)",5,3.175320
14,14,centro2030,Quais são algumas das áreas de intervenção do Programa Centro 2030?,"As áreas de intervenção incluem habitação, tecnologias críticas, defesa e segurança, além de garantir a aplicação eficaz e transparente dos fundos europeus.","Algumas das áreas de intervenção do Programa Centro 2030 

You can explicitly say:

Increasing k:

↑ Recall

↓ Precision

↑ Groundedness (usually)

Better chunking:

↑ Precision

↑ Groundedness

Larger chunks:

↑ Recall

↓ Precision

These trade-offs show deep understanding.